In [ ]:
#Uses python3

import numpy as np
import random as rnd
import time
import pandas as pd
import sys

In [17]:
#====================== all functions ======================
def gen_e0(n):
    """генерирует все возможные последовательности из 10 спинов"""
    n_b = str(bin(n))[2:]
    n_arr = [-1]*(10-len(n_b)) + [int(x) if x=='1' else -1 for x in n_b]
    return n_arr
def calc_e_dict(arr):
    """вычисляет энергию конкретной конфигурации"""
    J = 1
    e = 0
    e += (
        arr[0]*arr[1] + arr[0]*arr[2] + arr[0]*arr[3] + arr[0]*arr[4] +
        arr[5]*arr[6] + arr[5]*arr[7] + arr[5]*arr[8] + arr[5]*arr[9] +
        arr[0]*arr[1]*arr[5]*arr[6] +
        arr[0]*arr[2]*arr[5]*arr[7] + 
        arr[0]*arr[3]*arr[5]*arr[8] + 
        arr[0]*arr[4]*arr[5]*arr[9]
    )
    return -J*e
def gen_e_dict():
    """генерируем и заоплняем словарь всех энергий"""
    d = dict()
    for i in range(0,1024):
        conf = gen_e0(i)
        d[str(conf)] = calc_e_dict(conf)
    return d
#################################################################
def gen_state():
    """generate random start state with lenght L*L and [-1,1] components"""
    #np.random.seed(1234)
    state_s = np.random.choice([-1,1], L*L).reshape(L,L)
    state_t = np.random.choice([-1,1], L*L).reshape(L,L)
    return state_s, state_t
def choose_lattice():
    """choose random lattice s or t
    s == 1, t == -1"""
    return rnd.choice([1,-1])
def get_ij():
    """get spin for MC"""
    i,j = rnd.randint(0, L-1), rnd.randint(0, L-1)
    if i==L-1: i=-1
    if j==L-1: j=-1
    return i,j
def get_conf(i,j,s,t):
    """формируем конфигурацию, по вышеописанному правилу таблицы энергий"""
    arr = ([s[i,j]]+[s[i-1,j]]+[s[i,j+1]]+[s[i+1,j]]+[s[i,j-1]]+
           [t[i,j]]+[t[i-1,j]]+[t[i,j+1]]+[t[i+1,j]]+[t[i,j-1]])
    return arr
def mc_choice(dE,T):
    """принимаем или не принимаем переворот спина?"""
    if dE < 0:
        return True
    elif rnd.uniform(1,0) <= np.exp(-dE/T):
        return True
    else:
        return False 
#################################################################
def step(s,t,edict,T):
    """крутим 1 спин"""
    i,j = get_ij()
    arr = get_conf(i,j,s,t)
    if choose_lattice()==1:     ### решетка s
        arr_before = arr.copy() ### конфигурация ДО переворота спина
        arr[0] *= -1            ### конфигурация ПОСЛЕ переворота спина на решетке s
        dE = edict[str(arr)]-edict[str(arr_before)]
        if mc_choice(dE,T):
            s[i,j] *= -1
    else:                       ### решетка t
        arr_before = arr.copy() ### конфигурация ДО переворота спина
        arr[5] *= -1            ### конфигурация ПОСЛЕ переворота спина на решетке t
        dE = edict[str(arr)]-edict[str(arr_before)]
        if mc_choice(dE,T):
            t[i,j] *= -1
    return s,t
def mc_step(s, t, edict, T):
    """perform 2*L*L steps MC to relax"""
    for _ in range(2*L*L):
        s,t = step(s,t,edict,T)
    return s, t
#################################################################
def expand(state):
    _a = np.concatenate((state[L-1].reshape(1,L), state), axis=0)   # add last line to TOP
    a = np.concatenate((_a, _a[:,0].reshape(L+1,1)), axis=1)    # add first line to RIGHT
    return a
def calc_e(s,t):
    s,t = expand(s), expand(t)
    e = 0
    for i in range(L):
        for j in range(L):
            e += s[i,j]*s[i+1,j] + t[i,j]*t[i+1,j] + s[i,j]*s[i+1,j]*t[i,j]*t[i+1,j] # right neighbour
            e += s[i,j]*s[i,j+1] + t[i,j]*t[i,j+1] + s[i,j]*s[i,j+1]*t[i,j]*t[i,j+1] # down neighbour
    return -e/(L*L)
def calc_m(s):
    m = 0
    for i in range(L):
        for j in range(L):
            m += s[i,j]
    return m/(L*L) 
#################################################################
def model_at(T,N_avg,N_mc=1,Relax=1000):
    """Моделируем АТ"""
    E, M_s, M_t = [], [], []
    edict = gen_e_dict()
    state_s, state_t = gen_state()
    
    #relax 10**3 times be4 AVG
    for __ in range(Relax):
            state_s, state_t = mc_step(state_s, state_t, edict, T)
    #AVG every N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            state_s, state_t = mc_step(state_s, state_t, edict, T)
        E += [calc_e(state_s,state_t,L)]
        M_s += [calc_m(state_s,L)]
        M_t += [calc_m(state_t,L)]
    
    return E, M_s, M_t

In [13]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); edict = gen_e_dict()

%timeit [mc_step(s,t,edict,1) for _ in range(10)]

4.83 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); edict = gen_e_dict()
print(calc_e(s,t))

for _ in range(10):
    mc_step(s,t, edict, 1)
print(calc_e(s,t))

-0.0556
-4.2288


In [ ]:
# py     -- 4.83 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)